In [25]:
// let inputFile = "example.txt"
let inputFile = "input.txt"
 
let readLines filePath = List.ofSeq(System.IO.File.ReadLines(filePath))

let parseStackLine line = 
    Seq.toList line
    |> List.chunkBySize 4
    |> List.map (fun r ->  List.item 1 r)

parseStackLine "    [D]    "

let addToStack stackToAdd stacks =
    let init = List.init (List.length stackToAdd) (fun _ -> [])
    let original = if stacks = [] then init else stacks
    stackToAdd
    |> List.mapi (fun i s -> if s = ' ' then (List.item i original)  else s :: (List.item i original))

// addToStack ['A'; 'B'; ' '] []
// addToStack ['E'; 'B'; ' '] [['A']; ['B']; ['C']]

let parseStacks stacks line = 
    let parsed = parseStackLine line
    addToStack parsed stacks

parseStacks [] "    [D]    "

let stacks = 
    readLines inputFile
    |> List.takeWhile (fun r -> not(r.StartsWith(" 1 ")))
    |> List.fold parseStacks []
    |> List.map (fun s -> List.rev s)

stacks

index,value
0,"[ P, L, M, N, W, V, B, H ]"
1,"[ H, Q, M ]"
2,"[ L, M, Q, F, G, B, D, N ]"
3,"[ G, W, M, Q, F, T, Z ]"
4,"[ P, H, T, M ]"
5,"[ T, G, H, D, J, M, B, C ]"
6,"[ R, V, F, B, N, M ]"
7,"[ S, G, R, M, H, L, P ]"
8,"[ N, C, B, D, P ]"


In [26]:
open System.Text.RegularExpressions

let rx = Regex(@"move (\d+) from (\d+) to (\d+)", RegexOptions.Compiled)

let parseInstuructionsRow row =
    let matches = List.ofSeq(seq {
        for m in rx.Matches(row) do
            yield m
    })
    if List.length matches <> 1 then raise (System.ArgumentException($"Unexpected instructions {row}!"))
    let data = List.tail(List.ofSeq(seq { 
        for g in matches.Head.Groups do
            yield g.ToString().Trim()
    }))
    if List.length data <> 3 then raise (System.ArgumentException($"Unexpected instructions {row}!"))
    (List.item 0 data |> int, List.item 1 data |> int, List.item 2 data |> int)

parseInstuructionsRow "move 3 from 2 to 1"

let parseInstructions instructions line = 
    let parsed = parseInstuructionsRow line
    parsed :: instructions

parseInstructions [] "move 1 from 2 to 1"

let instructions = 
    readLines inputFile
    |> List.rev 
    |> List.takeWhile (fun r -> r <> "")
    |> List.fold parseInstructions []

instructions

index,Item1,Item2,Item3
0,8,3,2
1,1,9,5
2,5,4,7
3,6,1,4
4,8,6,8
5,8,4,5
6,4,9,5
7,4,7,9
8,7,7,2
9,4,5,2


In [28]:
let move stacks (amount, fromS, toS) = 
    let fromIndex = fromS - 1
    let toIndex = toS - 1
    let stackFrom = List.item fromIndex stacks
    let stackTo = List.item toIndex stacks
    let (moving, newStackFrom) = List.splitAt amount stackFrom
    // let newStackTo = (List.rev moving) @ stackTo
    let newStackTo = moving @ stackTo
    stacks
    |> List.updateAt fromIndex newStackFrom
    |> List.updateAt toIndex newStackTo

move [['A'; 'C']; ['D']] (2, 1, 2) 

let stacksAfter = instructions |> List.fold move stacks

let topStack = stacksAfter |> List.map (fun s -> List.head s)

System.String.Concat(Array.ofList(topStack))

GMPMLWNMG